In [2]:
import os
import pyspark
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
### path
GCS_CONNECTOR_SPARK = os.environ.get("GCS_CONNECTOR_SPARK")
GOOGLE_APPLICATION_CREDENTIALS = os.environ.get("GOOGLE_APPLICATION_CREDENTIALS")

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('Project-DE') \
    .set("spark.jars", GCS_CONNECTOR_SPARK) \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", GOOGLE_APPLICATION_CREDENTIALS)
    
sc = SparkContext(conf=conf)

sc._jsc.hadoopConfiguration().set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
sc._jsc.hadoopConfiguration().set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
sc._jsc.hadoopConfiguration().set("fs.gs.auth.service.account.json.keyfile", GOOGLE_APPLICATION_CREDENTIALS)
sc._jsc.hadoopConfiguration().set("fs.gs.auth.service.account.enable", "true")

your 131072x1 screen size is bogus. expect trouble
25/03/27 01:21:48 WARN Utils: Your hostname, DESKTOP-Q5PPUKU resolves to a loopback address: 127.0.1.1; using 172.30.227.126 instead (on interface eth0)
25/03/27 01:21:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/03/27 01:21:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
import gcsfs
import os
import dlt
import pyarrow.parquet as pq
import pyspark
from pyspark.sql import SparkSession, types
from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from google.cloud import bigquery

SCHEMAS = {
        "customers": types.StructType([
                        types.StructField('customer_id', types.StringType(), True), 
                        types.StructField('customer_unique_id', types.StringType(), True), 
                        types.StructField('customer_zip_code_prefix', types.IntegerType(), True), 
                        types.StructField('customer_city', types.StringType(), True), 
                        types.StructField('customer_state', types.StringType(), True)
                    ]),
        "geolocation": types.StructType([
                        types.StructField('geolocation_zip_code_prefix', types.IntegerType(), True), 
                        types.StructField('geolocation_lat', types.DecimalType(18, 15), True), 
                        types.StructField('geolocation_lng', types.DecimalType(18, 15), True), 
                        types.StructField('geolocation_city', types.StringType(), True), 
                        types.StructField('geolocation_state', types.StringType(), True)
                    ]),
        "order_items": types.StructType([
                        types.StructField('order_id', types.StringType(), True), 
                        types.StructField('order_item_id', types.IntegerType(), True), 
                        types.StructField('product_id', types.StringType(), True), 
                        types.StructField('seller_id', types.StringType(), True), 
                        types.StructField('shipping_limit_date', types.TimestampType(), True),
                        types.StructField('price', types.DoubleType(), True),
                        types.StructField('freight_value', types.DoubleType(), True)
                    ]),
        "order_payments": types.StructType([
                        types.StructField('order_id', types.StringType(), True), 
                        types.StructField('payment_sequential', types.IntegerType(), True), 
                        types.StructField('payment_type', types.StringType(), True), 
                        types.StructField('payment_installments', types.IntegerType(), True), 
                        types.StructField('payment_value', types.DoubleType(), True)
                    ]),
        "order_reviews": types.StructType([
                        types.StructField('review_id', types.StringType(), True), 
                        types.StructField('order_id', types.StringType(), True), 
                        types.StructField('review_score', types.IntegerType(), True), 
                        types.StructField('review_comment_title', types.StringType(), True), 
                        types.StructField('review_comment_message', types.StringType(), True),
                        types.StructField('review_creation_date', types.TimestampType(), True),
                        types.StructField('review_answer_timestamp', types.TimestampType(), True)
                    ]),
        "orders": types.StructType([
                        types.StructField('order_id', types.StringType(), True), 
                        types.StructField('customer_id', types.StringType(), True), 
                        types.StructField('order_status', types.StringType(), True), 
                        types.StructField('order_purchase_timestamp', types.TimestampType(), True), 
                        types.StructField('order_approved_at', types.TimestampType(), True),
                        types.StructField('order_delivered_carrier_date', types.TimestampType(), True),
                        types.StructField('order_delivered_customer_date', types.TimestampType(), True),
                        types.StructField('order_estimated_delivery_date', types.TimestampType(), True)
                    ]),
        "products": types.StructType([
                        types.StructField('product_id', types.StringType(), True), 
                        types.StructField('product_category_name', types.StringType(), True), 
                        types.StructField('product_name_lenght', types.IntegerType(), True), 
                        types.StructField('product_description_lenght', types.IntegerType(), True), 
                        types.StructField('product_photos_qty', types.IntegerType(), True),
                        types.StructField('product_weight_g', types.IntegerType(), True),
                        types.StructField('product_length_cm', types.IntegerType(), True),
                        types.StructField('product_height_cm', types.IntegerType(), True),
                        types.StructField('product_width_cm', types.IntegerType(), True)
                    ]),
        "sellers": types.StructType([
                        types.StructField('seller_id', types.StringType(), True), 
                        types.StructField('seller_zip_code_prefix', types.IntegerType(), True), 
                        types.StructField('seller_city', types.StringType(), True), 
                        types.StructField('seller_state', types.StringType(), True)
                    ]),
        "product_category_name_translation": types.StructType([
                        types.StructField('product_category_name', types.StringType(), True), 
                        types.StructField('product_category_name_english', types.StringType(), True)
                    ]),
        "olist_marketing_qualified_leads_dataset": types.StructType([
                        types.StructField('mql_id', types.StringType(), True),
                        types.StructField('first_contact_date', types.TimestampType(), True),
                        types.StructField('landing_page_id', types.StringType(), True),
                        types.StructField('origin', types.StringType(), True)
                    ]),
        "olist_closed_deals_dataset": types.StructType([
                        types.StructField('mql_id', types.StringType(), True),
                        types.StructField('seller_id', types.StringType(), True),
                        types.StructField('sdr_id', types.StringType(), True),
                        types.StructField('sr_id', types.StringType(), True),
                        types.StructField('won_date', types.TimestampType(), True),
                        types.StructField('business_segment', types.StringType(), True),
                        types.StructField('lead_type', types.StringType(), True),
                        types.StructField('lead_behaviour_profile', types.StringType(), True),
                        types.StructField('has_company', types.StringType(), True),
                        types.StructField('has_gtin', types.StringType(), True)
                    ])
        }

credential_path = os.environ.get("GOOGLE_APPLICATION_CREDENTIALS")
if credential_path:
    with open(credential_path, 'r') as file:
        os.environ["DESTINATION__BIGQUERY__CREDENTIALS"] = file.read()
else:
    raise Exception("GOOGLE_APPLICATION_CREDENTIALS is not set as environment variable")

pipeline = dlt.pipeline(
    pipeline_name='olist_ingestion_db',
    destination='bigquery',
    dataset_name='sources_olist'
)

spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()
    
def olist(type, specific_type=None):
    if type in ['customers', 'geolocation', 'orders', 'products', 'sellers']:
        df = spark.read \
            .option('header', 'true') \
            .schema(SCHEMAS.get(type)) \
            .csv(f'gs://nozpera_bucket/olist_{type}_dataset.csv')
        df.repartition(4).write.parquet(f'./parquet/{type}/')
        print(f'Process File: olist_{type}_dataset.csv')
        yield pq.read_table(f'./parquet/{type}/')
    elif type == 'order':
        df = spark.read \
            .option('header', 'true') \
            .schema(SCHEMAS.get(f'{type}_{specific_type}')) \
            .csv(f'gs://nozpera_bucket/olist_{type}_{specific_type}_dataset.csv')
        df.repartition(4).write.parquet(f'./parquet/{type}_{specific_type}/')
        print(f'Process File: olist_{type}_{specific_type}_dataset.csv')
        yield pq.read_table(f'./parquet/{type}_{specific_type}/')
    else:
        df = spark.read \
            .option('header', 'true') \
            .schema(SCHEMAS.get(type)) \
            .csv(f'gs://nozpera_bucket/{type}.csv')
        df.repartition(4).write.parquet(f'./parquet/{type}/')
        print(f'Process File: {type}.csv')
        yield pq.read_table(f'./parquet/{type}/')
        
dataset_type = 'olist_closed_deals_dataset'
specific_dataset_type = None

if dataset_type in ['customers', 'geolocation', 'orders', 'products', 'sellers']:
    info = pipeline.run(olist(dataset_type), table_name=f"olist_{dataset_type}_dataset", write_disposition="replace", loader_file_format="parquet")
    print(info)
elif dataset_type == 'order':
    info = pipeline.run(olist(dataset_type, specific_dataset_type), table_name=f"olist_{dataset_type}_{specific_dataset_type}_dataset", write_disposition="replace", loader_file_format="parquet")
    print(info)
else:
    info = pipeline.run(olist(dataset_type), table_name=dataset_type, write_disposition="replace", loader_file_format="parquet")
    print(info)

25/03/27 01:25:23 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 14, schema size: 10
CSV file: gs://nozpera_bucket/olist_closed_deals_dataset.csv
2025-03-27 01:25:23,983|[WARNING]|1835|139778474610816|dlt|type_mapping.py|to_db_datetime_type:56|Column flags for timezone or precision are not yet supported in this destination. One or both of these flags were used in column 'review_creation_date'. in table 'olist_order_reviews_dataset'.
2025-03-27 01:25:23,984|[WARNING]|1835|139778474610816|dlt|type_mapping.py|to_db_datetime_type:56|Column flags for timezone or precision are not yet supported in this destination. One or both of these flags were used in column 'review_answer_timestamp'. in table 'olist_order_reviews_dataset'.
2025-03-27 01:25:23,984|[WARNING]|1835|139778474610816|dlt|type_mapping.py|to_db_datetime_type:56|Column flags for timezone or precision are not yet supported in this destination. One or both of the

Process File: olist_closed_deals_dataset.csv


2025-03-27 01:25:26,478|[WARNING]|1835|139778474610816|dlt|type_mapping.py|to_db_datetime_type:56|Column flags for timezone or precision are not yet supported in this destination. One or both of these flags were used in column 'won_date'. in table 'olist_closed_deals_dataset'.


Pipeline olist_ingestion_db load step completed in 15.65 seconds
1 load package(s) were loaded to destination bigquery and into dataset sources_olist
The bigquery destination used zoomcamp@data-engineering-zoomcamp-arp.iam.gserviceaccount.com@data-engineering-zoomcamp-arp location to store data
Load package 1743013519.9781337 is LOADED and contains no failed jobs
